In [52]:
import os
import sys
import subprocess

from from_root import from_root

sys.path.insert(0, str(from_root("src")))

from utils import get_base_location
from read_and_write_docs import read_jsonl

In [53]:

corpus      = "Wiki"
data_type   = "training"

# Set NAS so can run on Windows laptop seamlessly
nas_base_loc = get_base_location()

known_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}/known_raw.jsonl"
unknown_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}/unknown_raw.jsonl"
metadata_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/metadata.rds"
model_loc = f"{nas_base_loc}/models/Qwen 2.5/Qwen2.5-0.5B-Instruct"
save_loc = f"{nas_base_loc}/paraphrase examples"

corpus      = "Wiki"
data_type   = "training"

openai_model = "gpt-4.1"
max_tokens   = 5000
temperature  = 0.7
n_samples    = 10

script_loc = str(from_root("scripts", "run_openai_paraphrase_method.py"))

In [54]:
# Load the pre-made problem datasets for speed
problem_dataset_base = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}"
problem_dataset_agg = read_jsonl(f"{problem_dataset_base}/{corpus}_{data_type}_agg.jsonl")

In [55]:
same_probs = problem_dataset_agg[problem_dataset_agg['known_author'] == problem_dataset_agg['unknown_author']].copy()
same_probs.sort_values(["highest_common_count"], ascending=[False], inplace=True)
same_probs[(same_probs['highest_common_count'] >= 3) & (same_probs['highest_common_count'] <= 10)].head(30)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
379,Greg_L vs Greg_L,Greg_L,Greg_L,greg_l_text_11,greg_l_text_10,9,", Ġthey Ġshould Ġhave Ġparticipated Ġin Ġthe Ġ..."
432,Haymaker vs Haymaker,Haymaker,Haymaker,haymaker_text_3,haymaker_text_2,9,"Ġat Ġthe Ġend Ġof Ġthe Ġday , Ġwe 're"
354,Fragments_of_Jade vs Fragments_of_Jade,Fragments_of_Jade,Fragments_of_Jade,fragments_of_jade_text_2,fragments_of_jade_text_10,8,"Ġme , Ġand Ġit 's Ġgetting Ġold .Ċ"
337,Fixentries vs Fixentries,Fixentries,Fixentries,fixentries_text_2,fixentries_text_5,8,Ġthe Ġindividual Ġher it ability Ġof Ġintellig...
248,DonaNobisPacem vs DonaNobisPacem,DonaNobisPacem,DonaNobisPacem,donanobispacem_text_5,donanobispacem_text_2,8,Ġafter Ġ 1 8 - 2 0 Ġweeks
168,Cla68 vs Cla68,Cla68,Cla68,cla68_text_2,cla68_text_1,8,i Ġagree Ġwith Ġnorth 8 0 0 0
176,Classicjupiter2 vs Classicjupiter2,Classicjupiter2,Classicjupiter2,classicjupiter2_text_4,classicjupiter2_text_5,7,Ġon Ġthe Ġsurreal ism Ġdiscussion Ġpage .Ċ
289,Equanimous1 vs Equanimous1,Equanimous1,Equanimous1,equanimous1_text_3,equanimous1_text_5,7,Ġthe Ġprofession Ġof Ġphysical Ġtherapy .Ċ for
68,Antidiskriminator vs Antidiskriminator,Antidiskriminator,Antidiskriminator,antidiskriminator_text_3,antidiskriminator_text_4,6,Ġthe Ġobject ivity Ġof Ġthe Ġreviews
392,Gwen_Gale vs Gwen_Gale,Gwen_Gale,Gwen_Gale,gwen_gale_text_3,gwen_gale_text_5,6,"Ġspeaking Ġonly Ġfor Ġmyself , Ġi"


In [56]:
diff_probs = problem_dataset_agg[problem_dataset_agg['known_author'] != problem_dataset_agg['unknown_author']].copy()
diff_probs.sort_values(["highest_common_count"], ascending=[False], inplace=True)
diff_probs[(diff_probs['highest_common_count'] >= 3) & (diff_probs['highest_common_count'] <= 10)].head(30)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
287,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_3,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
161,Caboga vs Chanakyathegreat,Caboga,Chanakyathegreat,caboga_text_5,chanakyathegreat_text_1,5,Ġ 2 0 0 8
262,Dweller vs Ecelan,Dweller,Ecelan,dweller_text_3,ecelan_text_2,5,.Ċ i Ġdon 't Ġthink
364,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_12,garda40_text_1,5,Ġ 2 0 1 1
365,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_2,garda40_text_1,5,Ġ 2 0 1 2
395,Gwen_Gale vs Habap,Gwen_Gale,Habap,gwen_gale_text_3,habap_text_1,5,", Ġbut Ġi Ġdon 't"
286,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_2,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
441,HeadleyDown vs Hipocrite,HeadleyDown,Hipocrite,headleydown_text_2,hipocrite_text_4,5,Ġon Ġthis Ġtalk Ġpage .Ċ
135,Britmax vs BrotherDarksoul,Britmax,BrotherDarksoul,britmax_text_1,brotherdarksoul_text_4,4,Ġis Ġno Ġneed Ġto
165,Chanakyathegreat vs Cla68,Chanakyathegreat,Cla68,chanakyathegreat_text_10,cla68_text_1,4,", Ġi Ġdon 't"


In [57]:
known_doc = "delicious_carbuncle_text_3"
unknown_doc = "dennis_brown_text_11"

In [58]:
env = dict(os.environ, PYTHONUNBUFFERED="1")
cmd = [
    sys.executable, "-u", script_loc,
    "--known_loc", known_loc,
    "--unknown_loc", unknown_loc,
    "--metadata_loc", metadata_loc,
    "--model_loc", model_loc,
    "--save_loc", save_loc,
    "--corpus", corpus,
    "--data_type", data_type,
    "--known_doc", known_doc,
    "--unknown_doc", unknown_doc,
    "--openai_model", openai_model,
    "--max_tokens", str(max_tokens),
    "--temperature", str(temperature),
    "--n", str(n_samples),
]

subprocess.run(cmd, text=True, env=env, check=True)

Working on problem: delicious_carbuncle_text_3 vs dennis_brown_text_11
Loading model
Loading data
Getting common n-grams
Generating paraphrases


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Scoring phrases
    Scoring known text
Processing Phrase - phrase_01
Processing Phrase - phrase_02
Processing Phrase - phrase_03
Processing Phrase - phrase_04
Processing Phrase - phrase_05
Processing Phrase - phrase_06
Processing Phrase - phrase_07
Processing Phrase - phrase_08
Processing Phrase - phrase_09
Processing Phrase - phrase_10
Processing Phrase - phrase_11
Processing Phrase - phrase_12
Processing Phrase - phrase_13
Processing Phrase - phrase_14
Processing Phrase - phrase_15
Processing Phrase - phrase_16
Processing Phrase - phrase_17
Processing Phrase - phrase_18
Processing Phrase - phrase_19
Processing Phrase - phrase_20
Processing Phrase - phrase_21
Processing Phrase - phrase_22
Processing Phrase - phrase_23
Processing Phrase - phrase_24
Processing Phrase - phrase_25
Processing Phrase - phrase_26
Processing Phrase - phrase_27
Processing Phrase - phrase_28
Processing Phrase - phrase_29
Processing Phrase - phrase_30
Processing Phrase - phrase_31
Processing Phrase - phrase_32
P

/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning:

CompletedProcess(args=['/Users/user/Documents/GitHub/av_distributions/my_venv/bin/python', '-u', '/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py', '--known_loc', '/Volumes/BCross/datasets/author_verification/training/Wiki/known_raw.jsonl', '--unknown_loc', '/Volumes/BCross/datasets/author_verification/training/Wiki/unknown_raw.jsonl', '--metadata_loc', '/Volumes/BCross/datasets/author_verification/training/metadata.rds', '--model_loc', '/Volumes/BCross/models/Qwen 2.5/Qwen2.5-0.5B-Instruct', '--save_loc', '/Volumes/BCross/paraphrase examples', '--corpus', 'Wiki', '--data_type', 'training', '--known_doc', 'delicious_carbuncle_text_3', '--unknown_doc', 'dennis_brown_text_11', '--openai_model', 'gpt-4.1', '--max_tokens', '5000', '--temperature', '0.7', '--n', '10'], returncode=0)